In [2]:
import pandas as pd

In [ ]:
asdf = pd.read_csv('../data/acts_sections.tar.gz')

In [6]:
asdf.head()

,ddl_case_id,act,section,bailable_ipc,number_sections_ipc,criminal
0,03-14-12-203600000492016,5250.0,NaN,NaN,NaN,1
1,11-06-03-202400002452018,4942.0,NaN,NaN,NaN,1
2,08-19-02-201000030712014,17698.0,NaN,NaN,1.0,1
3,17-21-18-206100036062014,14375.0,NaN,NaN,NaN,1
4,14-05-03-203700040542017,14692.0,NaN,NaN,NaN,1


In [9]:
asdf.describe()

,act,section,number_sections_ipc,criminal
count,7.680007e+07,7.103558e+07,2.094575e+07,7.681136e+07
mean,1.095572e+04,7.382429e+05,2.578470e+00,3.973017e-01
std,5.923539e+03,5.041001e+05,1.971519e+00,4.893394e-01
min,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00
25%,4.925000e+03,2.220210e+05,1.000000e+00,0.000000e+00
50%,1.081800e+04,7.625680e+05,2.000000e+00,0.000000e+00
75%,1.769800e+04,1.111797e+06,4.000000e+00,1.000000e+00
max,3.084900e+04,2.095512e+06,6.700000e+01,1.000000e+00
